In [261]:
import random
import numpy as np

In [262]:
class Set:
    def __init__(self, players, n = 10, winBonus = 10, roundBonus = 2, ordered = False, printMode = False):
        '''
        n (int) - number of rounds to be played per set
        players (1 x k numpy array) - numpy row slice of k player objects
        winBonus (int) - bonus awarded to the player who "takes" and therefore wins the Set
        roundBonus (int) - incremental bonus awarded for every round the set lasts
        k (int) - number of players for the set
        printMode (bool) - testing bool - used to print stuff out while game is going to make it look good
        
        
        Initialized later:
        
        playHist (k x n numpy array) - records actions for each player at every played round
        set_scores (k x 1 numpy array) - contains the calculated score for each player after set is played
        winner (int) - index of the player who "takes" and wins the set
        r_end (int) - round where a player decided to "take"
        plays_total (int) - the total number of actions taken in the set
        '''
        self.n = n
        self.players = players
        self.winBonus = winBonus
        self.roundBonus = roundBonus
        self.k = len(players)
        self.printMode = printMode
        
        # shuffle player order if not specified
        if not ordered:
            random.shuffle(self.players)
            
        
    def initialize_playHistory(self):
        '''
        Initialize the k x n playHistory array that will record the action in round r for player i in row i column r
        
        All actions that do not happen as a result of the game ending are NaN  -- (playHist)
        
        Also initializes each player's index for the set - index is used as row i in playHistory and set_scores
        
        Used in -play-
        '''
        self.playHist = np.empty((self.k, self.n))
        self.playHist[:] = np.NaN
        
        # initialize row/index for each player
        for idx, player in enumerate(self.players):
            player.idx = idx
            
            
    def record_actions(self, r):
        '''
        Records the action for each player in the playHistory array for one round. A 0 for row i column r represents
        a decision of pass for player i while a 1 represents a Take.
        
        Used in -play-
        '''
        
        for player in self.players:
            #decision = player.manual_decision(r = r, n = self.n, k = self.k)
            decision = player.rando_decision()
            if(decision == 'Pass'):
                self.playHist[player.idx][r] = 0
            elif(decision == 'Take'):
                self.playHist[player.idx][r] = 1
                return player.idx
            
        return None
    
        
    def play(self):
        '''
        Plays through the set and records winner, r_end, and plays_total
        '''
        self.initialize_playHistory()
        
        # play through this n rounds
        for r in range(self.n):
            
            if(self.printMode):
                print("==============================")
            
            result = self.record_actions(r)
            
            # result will return as an integer of the player's index who decided to "take", or None if all players "pass"
            if(isinstance(result, int)):
                self.winner = result
                self.r_end = r
                self.plays_total = (r * self.k) + result + 1
                return
            
            # next round begins if no one decided to "take"
            
        # no one decided to "take" so r_end and winner do not exist
        self.winner = np.nan
        self.r_end = np.nan
        self.plays_total = self.n * self.k
        return
    
    
    def calculate_score(self):
        '''
        Returns a k x 1 array of the payoff for each player i -- (set_scores)
        '''
        self.set_scores = np.zeros((self.k, 1))
        
        # multiply number of rounds completed (r_end) by round bonus, r_end will be nan if no one "takes" 
        if(~np.isnan(self.r_end)):
            self.set_scores += self.r_end * self.roundBonus
        else:
            self.set_scores += self.n * self.roundBonus
            
        if(~np.isnan(self.winner)):
            self.set_scores[self.winner] += self.winBonus
            
        return
    
    def run_set(self):
        '''
        Runs play and calculate scores to populate playHist and set_scores, returns total 
        '''
        self.play()
        self.calculate_score()
        
        return

In [263]:
class Player:
    def __init__(self):
        '''
        idx (int) - the index assigned to this player during a set, used to match the player to a row in the records
        setplayed (int) - the jth set that this player is a part of - assigned during gameplay
        score (int) - the score of this player after their set - assigned during gameplay
        '''
        self.idx = np.nan
        self.setplayed = np.nan
        self.score = np.nan
       
    def manual_decision(self, r, n, k):
        '''
        User manually inputs the decision to "take" or "pass", 1 is "take", 0 is "pass"
        User is given the history of plays, round number, total of rounds, total number of players
        '''
        print('Round {} of {}'.format(r + 1, n))
        print('You are in position {} out of {}'.format(self.idx + 1, k))
        print()
        
        decision = input('1 is Take, 0 is Pass: ')
        print()
        while True:
            if decision == '0':
                return 'Pass'
            elif decision == '1':
                return 'Take'
            else:
                decision = input('Error: 1 is Take, 0 is Pass: ')
                print()
                
    def rando_decision(self):
        b = random.random()
        if(b > 0.9):
            return 'Take'
        else:
            return 'Pass'
      

In [264]:
class Game:
    def __init__(self, allplayers, n = 10, mode = ('normal', {})):
        '''
        m (int) - number of sets to be played
        mode (tuple) - first element contains the mode for sets to be played, second element 
            is a dicionary with keys as keywords and values as arguments for the specific set
        allplayers (m x k numpy array) - m sets (rows) of k player objects (columns) per set
        n (int) - number of rounds per set to be played
        k (int) - number of players per set
                
        
        Initialized later:
        
        playHistorys (m x k x n numpy array) - contains m play histories
        sets_scores (m * k x 1 numpy array) - column array containing all scores
        gameSets(m x 1 numpy array) - column array containing all sets played out
        '''
        self.m = allplayers.shape[0]
        self.n = n
        self.allplayers = allplayers
        self.k = allplayers.shape[1]
        self.mode = mode
        
        
    def initialize_data(self):
        '''
        Initializes all data that will be stored in this Game object
        '''
        self.playHistorys = np.empty((self.m, self.k, self.n))
        self.sets_scores = np.empty((self.m * self.k, 1))
        self.playHistorys[:] = np.nan
        self.sets_scores[:] = np.nan
        self.gameSets = np.empty((self.m, 1), dtype = object)
        self.alltotalPlays = np.empty((self.m, 1))
        
    def run_sets(self):
        '''
        Runs m sets and computes playHistorys and scores and gameSets
        '''
        self.initialize_data()
        
        for j in range(self.m):
            
            # eventually change to something like Set.mode[0](players, **self.mode[1]) when there are different modes 
            jset = Set(self.allplayers[j], **self.mode[1])
            jset.run_set()
            self.playHistorys[j, :] = jset.playHist
            self.sets_scores[self.k * j:self.k * j+self.k] = jset.set_scores
            
            # record score and set j for each player
            for player in jset.players:
                player.setplayed = j
                player.score = jset.set_scores[player.idx]
            
            # add this set to the gameSets data
            self.gameSets[j] = jset
            # add totalPlays to alltotalPlays array
            self.alltotalPlays[j] = jset.plays_total

            
        return
            
        

In [265]:
def tester():
    theboys = np.array([Player() for i in range(10 * 3)]).reshape(10, 3)
    mygame = Game(theboys)
    mygame.run_sets()
    return mygame.playHistorys, mygame.sets_scores, mygame.alltotalPlays, mygame

In [266]:
a, b, c, d = tester()

In [267]:
a

array([[[ 0.,  0., nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0.,  1., nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[ 0.,  0., nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0.,  1., nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[ 0.,  0.,  0.,  0., nan, nan, nan, nan, nan, nan],
        [ 0.,  0.,  0.,  1., nan, nan, nan, nan, nan, nan],
        [ 0.,  0.,  0., nan, nan, nan, nan, nan, nan, nan]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., nan, nan, nan]],

       [[ 0.,  1., nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[ 0.,  0.,  1., nan, nan, nan, nan, nan, nan, nan],
        [ 0.,  0., nan, nan, n

In [268]:
b

array([[ 2.],
       [12.],
       [ 2.],
       [ 2.],
       [12.],
       [ 2.],
       [ 6.],
       [16.],
       [ 6.],
       [24.],
       [14.],
       [14.],
       [12.],
       [ 2.],
       [ 2.],
       [14.],
       [ 4.],
       [ 4.],
       [16.],
       [ 6.],
       [ 6.],
       [18.],
       [ 8.],
       [ 8.],
       [20.],
       [20.],
       [20.],
       [ 4.],
       [14.],
       [ 4.]])

In [269]:
c

array([[ 5.],
       [ 5.],
       [11.],
       [22.],
       [ 4.],
       [ 7.],
       [10.],
       [13.],
       [30.],
       [ 8.]])

In [270]:
d